# What happened with the most unloved colors?

*Author's Note: This analysis will be comparing pyspark to duckdb, thus each query will be done using duckdb then repeated with pyspark.

First, **what were the least used colors in the r/place 2022 canvas?**<br>
Let us classify the 'most unloved colors' as the 5 least used colors. What are those colors?

In [11]:
import duckdb

conn = duckdb.connect()

query = """
    WITH colors AS (
        SELECT
            pixel_color,
            COUNT(DISTINCT user_uniq_id) AS count
        FROM '../2022_place_canvas_history.parquet'
        GROUP BY pixel_color
    )
    SELECT pixel_color, count
    FROM colors
    ORDER BY count
    LIMIT 5
    """
    
res = conn.execute(query).df()
print(f"Top 10 painted pixels:\n{res}\n")

Top 10 painted pixels:
   pixel_color   count
0   Light teal  160380
1         Teal  190675
2  Pale purple  247720
3     Lavender  281712
4    Dark teal  322731



In [ ]:
import os
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("rplaceQuery").getOrCreate()
    
df = spark.read.parquet("../2022_place_canvas_history.parquet")
df.createOrReplaceTempView("canvas")

query = """
    WITH colors AS (
        SELECT
            pixel_color,
            COUNT(DISTINCT user_uniq_id) AS count
        FROM canvas
        GROUP BY pixel_color
    )
    SELECT pixel_color, count
    FROM colors
    ORDER BY count
    LIMIT 5
    """

result_df = spark.sql(query)
result_df.show()

spark.stop()

### Why did everyone hate teal and purple???

The least used colors during the event were all of the teal options and two purple colors. 
It is important to note that each of these were among the last couple of rounds of colors to be added later in the event. Dark teal and teal were added on day 2, and light teal, lavender, and pale purple added on day 3. However, even if we only look at the rankings during the time when all colors were available, the least loved colors stay in their rankings.

Before we can clearly explain why people did *not* use something, let's see why they did...

### What did users paint with the most unloved colors?

In [ ]:
import image as img

query = """
    SELECT pixel_color, coordinate, timestamp
    FROM 'userless_place_canvas.parquet'
    WHERE timestamp < '2022-04-04 22:47:40'
        AND pixel_color IN ('Light teal', 'Dark teal', 'Teal')
    """
    
img.process_img(query)

**Teal Canvas**

Note: Since the teal colors are very similar, they were grouped together.
<div>
<img src="teal-lovers.png" width="1000"/>
</div>

Here, there was quite a bit of teal scattered throughout the canvas, but there are some clear images that are forming...

For example, here is one very clear blob in the canvas...
<div>
<img src="teal-totoro.png"/>
</div>

...that actually turned out to be a part of an image of Totoro from Studio Ghibli's "My Neighbor Totoro". Here is a rendering on Mon, 04 Apr 2022 at 06:00:00 GMT
<div>
<img src="full-totoro.png"/>
</div>

Similarly, the very large square blob on the right side...
<div>
<img src="teal-freak.png"/>
</div>

...turned out to be an attack on the NoPixel Grand Theft Auto role-play community. Users attempted to cover their art with "FREAK SQUAD". Here is the progress as of Mon, 04 Apr 2022 at 15:00:00 GMT
<div>
<img src="full-freak.png"/>
</div>

Though there are many more examples of teal images being created, these are some of the most noticable ones. Unfortunately, both of these images are very quickly covered with other colors, adding to the dislike shown towards the three teals.

In [13]:
import image as img

query = """
    SELECT pixel_color, coordinate, timestamp
    FROM 'userless_place_canvas.parquet'
    WHERE timestamp < '2022-04-04 22:47:40'
        AND pixel_color IN ('#E4ABFF')
    """
    
img.process_img(query)

**Pale Purple Canvas**
<div>
<img src="pale-purple-lovers.png" width="1000"/>
</div>

None of the pixels here lead to some obvious image. Of course it is important to note the large number of placements in the bottom left corner, but considering that was France's territory and given the battles in that location, that is not surprising.

In [14]:
query = """
    SELECT pixel_color, coordinate, timestamp
    FROM 'userless_place_canvas.parquet'
    WHERE timestamp < '2022-04-04 22:47:40'
        AND pixel_color IN ('#94B3FF')
    """
    
img.process_img(query)

**Lavender Canvas**
<div>
<img src="lavender-lovers.png" width="1000"/>
</div>
Similar to teal, there are quite a few areas that look promising.



As for why so many chose other colors to paint the canvas with, the simple answer is the appeal of the colors. <br>
For teal, there were multiple other blue and green hues that were available for longer, and are generally considered more visually appealling.<br>
And as for the purples, they both often appeared as "supporting" or background colors rather than the main event.

### Pyspark vs DuckDB
DuckDB was so much nicer to use. Pyspark took too long to install and configure compared to DuckDB, and the query that I ran with it took much longer. Pyspark does not have an obvious advantage as both libaries have similar functionalities. Given I only compared the two with one query, I cannot say too much about it. However, solely given the long setup time I will not be returning to Pyspark unless it is necessary.